# Simple Code Agent
Perform login to HuggingFace Hub.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Playlist
Search for music recommendation to build a party playlist on DuckDuckGo.

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

## Menu
Create a menu to prepare.

In [ ]:
from smolagents import CodeAgent, tool, HfApiModel


# define a custom tool that suggests the menu based on the occasion
@tool
def suggest_menu(occasion: str) -> str:
    """Returns a menu suggestion based on the occasion.

    Args:
        occasion: str, the occasion name. Supported values are "casual", "formal",
            and "superhero".

    Returns:
        str, the suggested menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "custom menu for the butler."


# create the agent creating the menu
agent = CodeAgent(tools=[suggest_menu], model=HfApiModel())

# prepare the menu for the party
agent.run("Prepare a formal menu for the party.")

## Prepare everything for the party
Prepare everything for the party.

In [ ]:
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=["datetime"])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

## Upload agent
Push agent to hub

In [6]:
agent.push_to_hub("pierclgr/AlfredAgent")

README.md:   0%|          | 0.00/230 [00:00<?, ?B/s]

This agent has step_callbacks: they will be ignored by this method. 1

CommitInfo(commit_url='https://huggingface.co/spaces/pierclgr/AlfredAgent/commit/1df48a74243417c67cdcc6748f4c937d7e5a5f1d', commit_message='Upload agent', commit_description='', oid='1df48a74243417c67cdcc6748f4c937d7e5a5f1d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/pierclgr/AlfredAgent', endpoint='https://huggingface.co', repo_type='space', repo_id='pierclgr/AlfredAgent'), pr_revision=None, pr_num=None)

## Download agent
Download agent from HF Hub and execute it.

In [7]:
alfred_agent = agent.from_hub("pierclgr/AlfredAgent", trust_remote_code=True)
alfred_agent.run(
    "Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

agent.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

app.py:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

catering_service_tool.py:   0%|          | 0.00/938 [00:00<?, ?B/s]

requirements.txt:   0%|          | 0.00/50.0 [00:00<?, ?B/s]

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The playlist "Arkham Asylum (Batman, Joker, Gotham, Harley Quinn, Scarecrow, villains, rogues)" contains a
diverse range of songs that could create an enjoyable villain masquerade atmosphere. However, some songs may not be
suitable for a masquerade party, and we need to ensure the playlist mainly consists of energetic and ominous       
tracks.                                                                                                            
                                                                                                                   
I'll manually select a subset of songs that fit the theme and mood better for a party.                             
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
best_playlist = [                                                                                                  
    "PANIC!", "Smile", "Violence and Spiders", "Mad Hatter", "Pretty Pills for Broken Hearts",                     
    "Bad Guy", "Under My Skin", "Doctor", "Good Time", "Pink (Freak)", "Lunatics and Slaves",                      
    "No Place Like Home", "Imaginary Parties", "Let 'Em Talk", "Sun Goes Down", "World Class Fuck Up",             
    "Fuck With My Head", "The Takeover", "In My Head", "Black Sheep"                                               
]                                                                                                                  
final_answer(best_playlist)                                                                                        
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  best_playlist = [                                                                                                
      "PANIC!", "Smile", "Violence and Spiders", "Mad Hatter", "Pretty Pills for Broken Hearts",                   
      "Bad Guy", "Under My Skin", "Doctor", "Good Time", "Pink (Freak)", "Lunatics and Slaves",                    
      "No Place Like Home", "Imaginary Parties", "Let 'Em Talk", "Sun Goes Down", "World Class Fuck Up",           
      "Fuck With My Head", "The Takeover", "In My Head", "Black Sheep"                                             
  ]                                                                                                                
  final_answer(best_playlist)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['PANIC!', 'Smile', 'Violence and Spiders', 'Mad Hatter', 'Pretty Pills for Broken Hearts', 
'Bad Guy', 'Under My Skin', 'Doctor', 'Good Time', 'Pink (Freak)', 'Lunatics and Slaves', 'No Place Like Home', 
'Imaginary Parties', "Let 'Em Talk", 'Sun Goes Down', 'World Class Fuck Up', 'Fuck With My Head', 'The Takeover', 
'In My Head', 'Black Sheep']

[Step 4: Duration 14.86 seconds| Input tokens: 15,267 | Output tokens: 573]

['PANIC!',
 'Smile',
 'Violence and Spiders',
 'Mad Hatter',
 'Pretty Pills for Broken Hearts',
 'Bad Guy',
 'Under My Skin',
 'Doctor',
 'Good Time',
 'Pink (Freak)',
 'Lunatics and Slaves',
 'No Place Like Home',
 'Imaginary Parties',
 "Let 'Em Talk",
 'Sun Goes Down',
 'World Class Fuck Up',
 'Fuck With My Head',
 'The Takeover',
 'In My Head',
 'Black Sheep']

## Create party planning Agent
Create a complete party planning agent with many tools.

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool


@tool
def suggest_menu(occasion: str) -> str:
    """Returns a menu suggestion based on the occasion.

    Args:
        occasion (str): The occasion name. Supported values are "casual", "formal",
            and "superhero".

    Returns:
        str: The suggested menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."


@tool
def catering_service_tool(query: str) -> str:
    """Returns the highest rated catering service in Gotham City.

    Args:
        query (str): The search query.

    Returns:
        str: The highest rated catering service.
    """
    # example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7
    }

    # find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


class SuperheroPartyThemeTool(Tool):
    """
    A tool that suggests creative superhero-themed party ideas based on a category.

    Attributes:
        name (str): The name of the tool.
        description (str): A brief description of the tool.
        inputs (dict): A dictionary describing the input parameters.
        output_type (str): The type of the output.

    """

    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        """
        Generates a superhero-themed party idea based on the given category.

        Args:
            category (str): The type of superhero party.

        Returns:
            str: A unique party theme idea.
        """
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(),
                          "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")
    

agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
    ],
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To create the best playlist for a 'villain masquerade' party at Wayne's mansion, I will use the           
`superhero_party_theme_generator` tool to get a unique theme idea related to the 'villain masquerade' theme. Then, 
I will use a web search to find a suitable playlist based on this theme.                                           
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
theme_idea = superhero_party_theme_generator(category="villain masquerade")                                        
print("Generated theme idea:", theme_idea)                                                                         
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme_idea = superhero_party_theme_generator(category="villain masquerade")                                      
  print("Generated theme idea:", theme_idea)                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Generated theme idea: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the playlist contents, "gotham rogues" by BATSPOTIFY has 15 songs, while "the masquerade ball" by
goth d1ck! has 42 songs. Given the theme of a villain masquerade, the "the masquerade ball" playlist seems more    
fitting and diverse, with a wider range of music suitable for a masquerade-themed party.                           
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw")                                           
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw")                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw

[Step 4: Duration 11.34 seconds| Input tokens: 18,519 | Output tokens: 554]

'https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw'